# Metaheurísticas

## Lendo dados

In [16]:
def Read_data(path):
    
    import copy
    
    coords_x = []
    coords_y = []
    bacias = []
    nomes = []
    maturidades = []
    qualidades = []
    plays = []
    soterramentos = []
    pcgnas = []
    geracao = []
    migracao = []
    reservatorio = []
    geometria = []
    retencao = []
    pshc = []
    mc_vol = []
    mi_vol = []
    mc_vpl = []
    mi_vpl = []
    custos = []
    tempos_exec = []
    inicio_janela = []
    final_janela = []
    sondas_x = []
    sondas_y = []
    
    data = {
        1: coords_x,
        2: coords_y,
        3: bacias,
        4: nomes,
        5: maturidades,
        6: qualidades,
        7: plays,
        8: soterramentos,
        9: pcgnas,
        10: geracao,
        11: migracao,
        12: reservatorio,
        13: geometria,
        14: retencao,
        15: pshc,
        16: mc_vol,
        17: mi_vol,
        18: mc_vpl,
        19: mi_vpl,
        20: custos,
        21: tempos_exec,
        22: inicio_janela,
        23: final_janela,
        25: sondas_x,
        26: sondas_y
    }
    
    with open(path, 'r') as f:
        for i, line in enumerate(f):
            if i == 0:
                (n_projetos, custo_total) = int(line.split()[0]), float(line.split()[1])
            elif i == 24:
                n_sondas = int(line.split()[0])
            else:
                for elem in line.split('[')[1].split(']')[0].split(','):
                    try:
                        data[i].append(float(elem))
                    except:
                        data[i].append(elem)
    
    return n_projetos, n_sondas, custo_total, data


## Classe para lista duplamente encadeada

In [17]:
class Node:
    
    """
    Documentação:
    """
    
    def __init__(self, data=None):
        self.data = data
        self.next = None
        self.prev = None
    

class DoublyLinkedList:
    
    """
    Documentação:
    """
    
    def __init__(self, nodes=None):
        self.head = None
        if nodes is not None:
            node = Node(data=nodes.pop(0))
            self.head = node
            for elem in nodes:
                new_node = Node(data=elem)
                node.next = new_node
                new_node.prev = node
                node = node.next
    
    def __iter__(self):
        node = self.head
        while node is not None:
            yield node
            node = node.next
    
    def add_node_begining(self, value_to_add):
        new_node = Node(data=value_to_add)
        
        if not self.head:
            self.head = new_node
            return
        
        new_node.next = self.head
        self.head.prev = new_node
        self.head = new_node
    
    def add_node_end(self, value_to_add):
        
        new_node = Node(data=value_to_add)
        
        if not self.head:
            self.head = new_node
            return
        
        for curr in self:
            pass
        curr.next = new_node
        new_node.prev = curr
    
    def add_node_after_node(self, value_to_add, node_ref):
        new_node = Node(data=value_to_add)
        new_node.next = node_ref.next
        new_node.prev = node_ref
        node_ref.next.prev = new_node
        node_ref.next = new_node
    
    def add_node_before_node(self, value_to_add, node_ref):
        new_node = Node(data=value_to_add)
        if node_ref == self.head:
            self.head.prev = new_node
            new_node.next = self.head
            self.head = new_node
        else:
            node_ref.prev.next = new_node
            new_node.prev = node_ref.prev
            new_node.next = node_ref
            node_ref.prev = new_node
    
    def remove_node_by_ref(self, node):
        if node == self.head:
            node.next.prev = None
            self.head = node.next
            del node
        else:
            node.next.prev = node.prev
            node.prev.next = node.next
            del node
    
    def find_value(self, value):
        for node in self:
            if node.data == value: break
        return node

## Função para Pré Processamento

In [18]:
def PreProcessamento(dados, n_projetos, n_sondas, t_init, t_final, delta_t):
    
    import math
    import copy
    
    dados_local = copy.deepcopy(dados)
    
    n_periodos = ((t_final - t_init) // delta_t) + 1
    
    # convertendo dados de tempo para períodos -------------- TODO: verificar se não está somando 1 a mais
    for i in range(n_projetos):
        dados_local[21][i] = (dados_local[21][i] // delta_t) + 1
        dados_local[22][i] = (dados_local[22][i] // delta_t)
        dados_local[23][i] = (dados_local[23][i] // delta_t)
    
    # calculando tempo de deslocamento entre projetos
    desloc = []
    lag = n_sondas
    for i in range(n_projetos+n_sondas):
        desloc.append([])
        for j in range(n_projetos+n_sondas):
            if (i<n_sondas) and (j<n_sondas):
                dist = math.sqrt( (dados[25][i] - dados[25][j])**2 + (dados[26][i] - dados[26][j])**2 )
            elif (i<n_sondas) and not(j<n_sondas):
                dist = math.sqrt( (dados[25][i] - dados[1][j-lag])**2 + (dados[25][i] - dados[2][j-lag])**2 )
            elif not(i<n_sondas) and (j<n_sondas):
                dist = math.sqrt( (dados[1][i-lag] - dados[26][j])**2 + (dados[2][i-lag] - dados[26][j])**2 )
            else:
                dist = math.sqrt( (dados[1][i-lag] - dados[1][j-lag])**2 + (dados[2][i-lag] - dados[2][j-lag])**2 )
            
            # converte para períodos: --------------------- TODO : verificar se não está somando 1 a mais
            if delta_t == 1:
                desloc[i].append(dist)
            else:
                desloc[i].append((dist // delta_t) + 1)
    
    return dados_local, n_periodos, desloc

## Implementação da heurística de construção

In [19]:
def ConstruirSolucao(dados, n_projetos, n_sondas, n_periodos, custo_total, setup, criterio='lucro/custo'):
    
    import math
    import copy
    import numpy as np
    import heapq
    
    lag = n_sondas
    
    # inicializando solução
    s = {i:DoublyLinkedList(nodes=[[-1, 0, n_periodos]]) for i in range(n_sondas)}
    
    # inicializando lista de candidatos, por sonda
    s_candidatos = {i:[] for i in range(n_sondas)}
    
    # armazena lista de candidatos como uma heap, dependendo do critério guloso
    for i in range(n_sondas):
        for j in range(n_projetos):
            if (criterio == 'lucro'):
                criterio_val = dados[19][j]
                heapq.heappush( s_candidatos[i], (-criterio_val, j) )
            elif (criterio == 'lucro/custo'):
                criterio_val = dados[19][j] / dados[20][j]
                heapq.heappush( s_candidatos[i], (-criterio_val, j) )
            elif (criterio == 'lucro/duracao'):
                criterio_val = dados[19][j] / dados[21][j]
                heapq.heappush( s_candidatos[i], (-criterio_val, j) )
            elif (criterio == 'custo'):
                criterio_val = dados[20][j]
                heapq.heappush( s_candidatos[i], (criterio_val, j) )
            elif (criterio == 'duracao'):
                criterio_val = dados[21][j]
                heapq.heappush( s_candidatos[i], (criterio_val, j) )
    
    # inicializa contador de gastos total
    gastos = 0
    
    # inicializa conjunto de projetos já selecionados
    proj_usados = set()
    
    # conjunto de sondas é representado com uma Heap
    sondas = [(-n_periodos, i) for i in range(n_sondas)]
    
    # inicializa valor da função objetivo da solução
    fitness = 0
    
    # enquanto nenhuma restrição for alcançada
    while (gastos < custo_total and sondas):
        
        # escolher uma sonda
        (tam_prev, sonda) = heapq.heappop(sondas)
        
        # escolher um projeto
        projeto = heapq.heappop(s_candidatos[sonda])[1]
        
        # se valor do projeto não aumenta a função objetivo, pular
        if (dados[19][projeto] <= 0.):
            if s_candidatos[sonda] != []:
                heapq.heappush(sondas, (tam_prev, sonda))
            continue
        
        # se projeto já foi escolhido, pular
        if (projeto in proj_usados):
            if s_candidatos[sonda] != []:
                heapq.heappush(sondas, (tam_prev, sonda))
            continue
        
        # se executar o projeto excede limite de gastos, pular
        if (gastos + dados[20][projeto] > custo_total):
            if s_candidatos[sonda] != []:
                heapq.heappush(sondas, (tam_prev, sonda))
            continue
        
        aloc = False
        proc = dados[21][projeto]
        
        # percorre atividades da sonda para ver se existe janela disponível na sonda
        for i, node in enumerate(s[sonda]):
            
            # se node não for de janela de tempo, pular
            if (node.data[0] != -1):
                continue
            
            # calcula tempo de deslocamento / setup =====================
            # se node for head, considerar origem da sonda
            if node.prev == None:
                setup = desloc[sonda][projeto + lag]
            # senão, considerar projeto anterior
            else:
                setup = desloc[node.prev.data[0] + lag][projeto + lag]
            
            # calcula o mínimo tempo de início e o máximo tempo de término do projeto dentro da janela
            release = max(node.data[1], dados[22][projeto])
            due = min(node.data[2], dados[23][projeto])
            
            # se consigo colocar o projeto dentro da janela
            if (release + proc + setup <= due):
                
                # escolher tempo de inicio para alocar projeto
                delta1 = (due - release) - (proc + setup) + (node.data[2] - due)
                delta2 = (due - release) - (proc + setup) + (release - node.data[1])
                if (delta1 >= delta2):
                    inicio = copy.copy(release)
                    final = copy.copy(release + setup + proc)
                else:
                    inicio = copy.copy(due - proc - setup)
                    final = copy.copy(due)
                
                # alocar
                proj_usados.add(copy.copy(projeto))
                fitness += dados[19][projeto]
                aloc = True
                
                # atualizando a lista de atividades ========================================
                # se projeto preenche janela toda
                if ( (int(node.data[1])==int(inicio)) and (int(node.data[2])==int(final)) ):
                    
                    # substituir janela pela alocação do projeto
                    node.data = [copy.copy(projeto), inicio, final]
                
                # senão, se projeto preenche início da janela    
                elif (int(node.data[1])==int(inicio)):
                    
                    # atualizar release da janela
                    node.data[1] = final
                    
                    # inserir projeto antes da janela
                    s[sonda].add_node_before_node(value_to_add=[projeto, inicio, final], node_ref=node)
                
                # senão, se projeto preenche final da janela
                elif (int(node.data[2])==int(final)):
                    
                    # atualiza a due da janela
                    node.data[2] = inicio
                    
                    # inserir projeto depois da janela
                    s[sonda].add_node_after_node(value_to_add=[projeto, inicio, final], node_ref=node)
                
                # senão, se projeto preenche o meio da janela
                else:
                    
                    # atualiza release da janela
                    temp = copy.copy(node.data[1])
                    node.data[1] = final
                    
                    # insere janela nova antes da janela original
                    s[sonda].add_node_before_node(value_to_add=[-1, temp, inicio], node_ref=node)
                    
                    # insere projeto antes da janela original
                    s[sonda].add_node_before_node(value_to_add=[projeto, inicio, final], node_ref=node)
                    
                break
            else:
                # senão, tentar realocar projetos ====================================================
                
                # verificar se janela da sonda tem interceção com janela do projeto
                if ((node.data[1] < dados[23][projeto]) or (node.data[2] > dados[22][projeto])):
                    
                    # consigo antecipar o prev? quanto?
                    delta_prev = 0
                    if node.prev == None:
                        continue
                    elif node.prev.prev == None:
                        continue
                    else:
                        if node.prev.prev.data[0] != -1:
                            continue
                        else:
                            delta_prev = node.prev.data[1] - max(node.prev.prev.data[1], dados[22][node.prev.data[0]])
                    
                    # consigo postergar o next? quanto?
                    delta_next = 0
                    if node.next == None:
                        continue
                    if node.next.next == None:
                        continue
                    else:
                        if node.next.next.data[0] != -1:
                            continue
                        else:
                            delta_next = min(node.next.next.data[2], dados[23][node.next.data[0]]) - node.next.data[2]
                    
                    # quanto de delta eu preciso?
                    delta_min = release + proc + setup - due
                    
                    # só com antecipação do prev, torna viável?
                    if (delta_prev >= delta_min ):
                        # fazer antecipação do prev
                        node.prev.data[1] -= delta_min
                        node.prev.data[2] -= delta_min
                        node.prev.prev.data[2] -= delta_min
                        aloc = True
                        node.data[1] -= delta_min
                    # só com postergação do next, torna viável?
                    elif (delta_next >= delta_min):
                        # fazer postergação do next
                        node.next.data[1] += delta_min
                        node.next.data[2] += delta_min
                        node.next.next.data[1] += delta_min
                        aloc = True
                        node.data[2] += delta_min
                    # somando antecipação do prev e postergação do next, torna viável?
                    elif (delta_prev + delta_next >= delta_min):
                        # fazer antecipação do prev
                        node.prev.data[1] -= delta_prev
                        node.prev.data[2] -= delta_prev
                        node.prev.prev.data[2] -= delta_prev
                        node.data[1] -= delta_min
                        # fazer postergação do next
                        node.next.data[1] += (delta_min - delta_prev)
                        node.next.data[2] += (delta_min - delta_prev)
                        node.next.next.data[1] += (delta_min - delta_prev)
                        aloc = True
                        node.data[2] += delta_min
                        
                    # alocar projeto
                    if aloc:
                        node.data[0] = projeto
                        proj_usados.add(copy.copy(projeto))
                        fitness += dados[19][projeto]
                        break
        
        # se projeto foi alocado, atualizar dados
        if (aloc):
            gastos += dados[20][projeto]
            # se sonda ainda tem candidatos, colocar de volta na heap com valor atualizado
            if s_candidatos[sonda] != []:
                heapq.heappush(sondas, ( (tam_prev + (final-inicio+1)) , sonda))
        # senão, apenas recolocar a sonda
        else:
            if s_candidatos[sonda] != []:
                heapq.heappush(sondas, (tam_prev, sonda))
    
    return fitness, s, s_candidatos

In [20]:
def VerificaRestricaoOrcamento():
    
    """
    Documentação:
    """
    
    return

In [21]:
def VerificaRestricaoUnicidadeProjetos():
    
    """
    Documentação:
    """
    
    return

In [22]:
def VerificaRestricaoTimeWindows():
    
    """
    Documentação:
    """
    
    return

In [23]:
def VerificaRestricaoScheduling():
    
    """
    Documentação:
    """
    
    return

In [24]:
def VerificaRestricoes():
    
    """
    Documentação:
    """
    
    return

In [25]:
def RodarHeuristica():
    
    """
    Documentação:
    """
    
    return resultados

In [26]:
ls instancias/

instancia_100projetos_10sondas.dat  instancia_300projetos_10sondas.dat
instancia_100projetos_20sondas.dat  instancia_300projetos_20sondas.dat
instancia_100projetos_2sondas.dat   instancia_300projetos_2sondas.dat
instancia_100projetos_5sondas.dat   instancia_300projetos_5sondas.dat
instancia_10projetos_2sondas.dat    instancia_30projetos_2sondas.dat
instancia_200projetos_10sondas.dat  instancia_500projetos_10sondas.dat
instancia_200projetos_20sondas.dat  instancia_500projetos_20sondas.dat
instancia_200projetos_2sondas.dat   instancia_500projetos_2sondas.dat
instancia_200projetos_5sondas.dat   instancia_500projetos_5sondas.dat
instancia_20projetos_2sondas.dat    instancia_50projetos_2sondas.dat


In [27]:
arq = 'instancias/instancia_10projetos_2sondas.dat'

In [28]:
n_projetos, n_sondas, custo_total, data = Read_data(path=arq)

In [29]:
t_init, t_final, delta_t = 0, 5*12*4*7, 7*4*3

In [30]:
dados_local, n_periodos, desloc = PreProcessamento(data, n_projetos, n_sondas, t_init, t_final, delta_t)

In [31]:
%%time

c = 'lucro'
f, s, _ = ConstruirSolucao(dados_local, n_projetos, n_sondas, n_periodos, custo_total, setup=desloc, criterio=c)

print ('fitness: ', f)

for i in range(n_sondas):
    for node in s[i]:
        print (i, node.data)

fitness:  1936.1810158036349
0 [-1, 0, 14.0]
0 [6, 14.0, 17.0]
0 [5, 17.0, 20.0]
0 [-1, 20.0, 21]
1 [7, 0, 5.0]
1 [-1, 5.0, 16.0]
1 [4, 16.0, 20.0]
1 [-1, 20.0, 21]
CPU times: user 228 ms, sys: 73.2 ms, total: 301 ms
Wall time: 1.58 s


In [33]:
%%time

c = 'lucro/custo'
f, s, _ = ConstruirSolucao(dados_local, n_projetos, n_sondas, n_periodos, custo_total, setup=desloc, criterio=c)

print ('fitness: ', f)

for i in range(n_sondas):
    for node in s[i]:
        print (i, node.data)

fitness:  2023.9826987263687
0 [-1, 0, 13.0]
0 [9, 13.0, 17.0]
0 [5, 17.0, 20.0]
0 [-1, 20.0, 21]
1 [-1, 0, 1.0]
1 [6, 1.0, 4.0]
1 [-1, 4.0, 16.0]
1 [4, 16.0, 20.0]
1 [-1, 20.0, 21]
CPU times: user 5.32 ms, sys: 113 µs, total: 5.43 ms
Wall time: 3.48 ms


In [40]:
%%time

c = 'lucro/duracao'
f, s, _ = ConstruirSolucao(dados_local, n_projetos, n_sondas, n_periodos, custo_total, setup=desloc, criterio=c)

print ('fitness: ', f)

for i in range(n_sondas):
    for node in s[i]:
        print (i, node.data)

fitness:  2023.9826987263687
0 [-1, 0, 13.0]
0 [9, 13.0, 17.0]
0 [5, 17.0, 20.0]
0 [-1, 20.0, 21]
1 [-1, 0, 1.0]
1 [6, 1.0, 4.0]
1 [-1, 4.0, 16.0]
1 [4, 16.0, 20.0]
1 [-1, 20.0, 21]
CPU times: user 502 µs, sys: 2.93 ms, total: 3.43 ms
Wall time: 2.3 ms


In [47]:
%%time

c = 'custo'
f, s, _ = ConstruirSolucao(dados_local, n_projetos, n_sondas, n_periodos, custo_total, setup=desloc, criterio=c)

print ('fitness: ', f)

for i in range(n_sondas):
    for node in s[i]:
        print (i, node.data)

fitness:  1845.800170831143
0 [-1, 0, 12.0]
0 [1, 12.0, 17.0]
0 [5, 17.0, 20.0]
0 [-1, 20.0, 21]
1 [-1, 0, 1.0]
1 [6, 1.0, 4.0]
1 [-1, 4.0, 16.0]
1 [4, 16.0, 20.0]
1 [-1, 20.0, 21]
CPU times: user 1.97 ms, sys: 405 µs, total: 2.37 ms
Wall time: 1.28 ms


In [15]:
%%time

c = 'duracao'
f, s, _ = ConstruirSolucao(dados_local, n_projetos, n_sondas, n_periodos, custo_total, setup=desloc, criterio=c)

print ('fitness: ', f)

for i in range(n_sondas):
    for node in s[i]:
        print (i, node.data)

fitness:  1496.3633253243404
0 [-1, 0, 12.0]
0 [1, 12.0, 17.0]
0 [5, 17.0, 20.0]
0 [-1, 20.0, 21]
1 [-1, 0, 1.0]
1 [6, 1.0, 4.0]
1 [-1, 4.0, 16.0]
1 [3, 16.0, 20.0]
1 [-1, 20.0, 21]
CPU times: user 11.4 ms, sys: 4.31 ms, total: 15.7 ms
Wall time: 9.93 ms
